In [1]:
import cv2
import tensorflow
import numpy as np
import pandas as pd
import os

In [2]:
categories=os.listdir('C:/Users/hp/face expression recognition/data/train')
categories

['angry', 'disgust', 'fear', 'happy', 'nutral', 'sad', 'surprise']

In [3]:
img=cv2.imread('C:/Users/hp/face expression recognition/data/train/angry/5132.png')
img.shape

(48, 48, 3)

In [4]:
train_image=[]
train_label=[]
for cat in categories:
    train_img_path=os.path.join('C:/Users/hp/face expression recognition/data/train',cat)
    image_name=os.listdir(train_img_path)
    
    for image in image_name:
        img=cv2.imread(os.path.join(train_img_path,image))
        train_image.append(img)
        train_label.append(categories.index(cat))

In [5]:
train_image[28000].shape


(48, 48, 3)

In [6]:
test_image=[]
test_label=[]
for cat in categories:
    test_img_path=os.path.join('C:/Users/hp/face expression recognition/data/test',cat)
    image_name=os.listdir(test_img_path)
    
    for image in image_name:
        img=cv2.imread(os.path.join(test_img_path,image))
        test_image.append(img)
        test_label.append(categories.index(cat))

In [7]:
val_image=[]
val_label=[]
for cat in categories:
    val_img_path=os.path.join('C:/Users/hp/face expression recognition/data/val',cat)
    image_name=os.listdir(val_img_path)
    
    for image in image_name:
        img=cv2.imread(os.path.join(val_img_path,image))
        val_image.append(img)
        val_label.append(categories.index(cat))

In [8]:
train_img=np.reshape(train_image,(len(train_image),48,48,3))
test_img=np.reshape(test_image,(len(test_image),48,48,3))
val_img=np.reshape(val_image,(len(val_image),48,48,3))
train_lab=np.reshape(train_label,(len(train_label),))
test_lab=np.reshape(test_label,(len(test_label),))
val_lab=np.reshape(val_label,(len(val_label),))

In [9]:
train_img.shape

(28709, 48, 48, 3)

In [10]:
train_lab.shape

(28709,)

In [11]:
from sklearn.utils import shuffle    

In [12]:
train_img,train_lab=shuffle(train_img,train_lab)
test_img,test_lab=shuffle(test_img,test_lab)
val_img,val_lab=shuffle(val_img,val_lab)
train_lab[0]

0

In [13]:
from tensorflow.keras.utils import to_categorical
train_lab=to_categorical(train_lab,num_classes=7,dtype='int')
test_lab=to_categorical(test_lab,num_classes=7,dtype='int')
val_lab=to_categorical(val_lab,num_classes=7,dtype='int')
train_lab

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Dropout,Input,Flatten

In [22]:
expression=Sequential()
expression.add(Input(shape=(48,48,3)))

expression.add(Conv2D(64,3,activation='relu',padding='same'))
expression.add(BatchNormalization())
expression.add(Conv2D(64,3,activation='relu'))
expression.add(MaxPooling2D(pool_size=2))
expression.add(Dropout(0.2))

expression.add(Conv2D(128,3,activation='relu'))
expression.add(BatchNormalization())
expression.add(Conv2D(128,3,activation='relu'))
expression.add(MaxPooling2D(pool_size=2))
expression.add(Dropout(0.2))

expression.add(Flatten())
expression.add(Dense(56,activation='relu'))
expression.add(Dense(28,activation='relu'))
expression.add(Dense(7,activation='softmax'))

expression.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
expression.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 48, 64)        1792      
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
batch_normalization_3 (Batch (None, 21, 21, 128)      

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [24]:
checkpoint=ModelCheckpoint('model-{epoch:03d}.model',monitor='val_accuracy',save_best_only=True)

In [25]:
history=expression.fit(train_img,train_lab,validation_data=(val_img,val_lab),epochs=30,callbacks=[checkpoint])

Epoch 1/30
898/898 [==============================] - ETA: 0s - loss: 1.6739 - accuracy: 0.3201WARNING:tensorflow:From C:\Users\hp\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model-001.model\assets
898/898 [==============================] - 416s 463ms/step - loss: 1.6739 - accuracy: 0.3201 - val_loss: 1.5452 - val_accuracy: 0.3734
Epoch 2/30
898/898 [==============================] - 418s 465ms/step - loss: 1.4207 - accuracy: 0.4206 - val_loss: 1.4813 - val_accuracy: 0.3920
Epoch 3/30
898/898 [==============================] - 484s 539ms/step - los

In [26]:
_,accuracy=expression.evaluate(test_img,test_lab)

113/113 [==============================] - 7s 66ms/step - loss: 1.9799 - accuracy: 0.5924
